# Bug triage with Deep Learning - DWEN

In [1]:
import keras

Using TensorFlow backend.


In [2]:
from __future__ import print_function, division

In [3]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [4]:
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, \
    GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras.layers.advanced_activations import LeakyReLU, ELU
from keras import optimizers

In [5]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

## Auxiliary methods

## Configurações Globais

In [6]:
MAX_SEQUENCE_LENGTH_T = 20 # 40
MAX_SEQUENCE_LENGTH_D = 100 # 200
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000

### Parse bugs preproprecessed

In [7]:
# Domain to use
DOMAIN = 'openoffice'
METHOD = 'baseline_dwen'
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Path embeddings
EMBED_DIR='data/embed'
# Save model
SAVE_PATH = '{}_feature@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_feature_@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [8]:
baseline = Baseline(DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

In [9]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

Creating the buckets...


#### Loading bug ids in memory

In [10]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


98070

#### Dicionário de títulos e descrições

In [11]:
%%time

experiment.load_bugs()
len(baseline.sentence_dict)


CPU times: user 11.8 s, sys: 858 ms, total: 12.7 s
Wall time: 12.6 s


#### Hashing bugs by buckets

In [12]:
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare the train and test

In [13]:
experiment.prepare_dataset(issues_by_buckets, path_train='train_chronological', path_test='test_chronological')
# Read and create the test queries duplicates
retrieval.create_queries()

Reading train data
Reading bug ids


In [14]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

In [15]:
idx = np.random.choice(baseline.bug_ids, 1)[0]
baseline.bug_set[idx]

{'bug_severity': '4\n',
 'bug_status': '2\n',
 'component': '129\n',
 'creation_ts': '2007-12-13 15:42:00 +0000',
 'delta_ts': '2013-08-07 14:43:03 +0000',
 'description': 'the fix of issue lets a wrong string in the undo menu appear open the organization and formatting window if not already open switch to list organization right click organization modify change anything here for example apply a bullet the undo menu shows insert organization it should be change numbering style style name',
 'description_word': array([   2,  402,   11,   93, 3108,    4,  171,  231,    6,    2,  744,
         145,  436,   40,    2,    3,    9,  468,  131,   42,   16,  451,
          40,  717,    5,  147,    3,  143,  102,    3,  977,  146,  842,
         317,   18,  214,  814,    4, 1092,    2,  744,  145,  370,  161,
           3,   15,   65,   25,  146,  448,  167,  167,   67,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    

### Generating the batch test

In [16]:
"Train ", len(baseline.dup_sets_train)

('Train ', 12268)

In [17]:
%%time

batch_size = 64
batch_size_test = 128

# we want a constant validation group to have a frame of reference for model performance
batch_triplets_valid, valid_input_sample, valid_input_pos, valid_input_neg, valid_sim = baseline.batch_iterator(baseline.train_data, 
                                                                                          baseline.dup_sets_train,
                                                                                          bug_train_ids,
                                                                                          batch_size_test, 1)

valid_title_sample_a = np.concatenate([valid_input_sample['title'], valid_input_sample['title']], 0)
valid_title_sample_b = np.concatenate([valid_input_pos['title'], valid_input_neg['title']], 0)
valid_desc_sample_a = np.concatenate([valid_input_sample['description'], valid_input_sample['description']], 0)
valid_desc_sample_b = np.concatenate([valid_input_pos['description'], valid_input_neg['description']], 0)

test_gen = ([valid_title_sample_a, valid_title_sample_b, valid_desc_sample_a, valid_desc_sample_b], valid_sim)

# Max sequence title
MAX_SEQUENCE_LENGTH_T = valid_input_sample['title'].shape[1]
MAX_SEQUENCE_LENGTH_D = valid_input_sample['description'].shape[1]

CPU times: user 34.7 ms, sys: 0 ns, total: 34.7 ms
Wall time: 34.2 ms


In [18]:
valid_input_sample['title'].shape, valid_input_sample['description'].shape, valid_sim.shape

((128, 20), (128, 100), (128,))

### Validar entrada

In [19]:
%%time 

baseline.display_batch(baseline.train_data, baseline.dup_sets_train, bug_train_ids, 5)

***Title***: organization is missing
***Title***: person do not import correcty
***Description***: look at url above autoshape is missing on ooo
***Description***: person do not import correctly in this document from plan maker competitive comparison http www softmaker de pmwcomp en htm
***similar = 1
########################
***Title***: basing autofiltering on the result set not all rows
***Title***: list of choices from country does not update
***Description***: this is another behavioural difference between oo and product that you may or may not wish to emulate the behaviour relates to autofiltering take as an example a sheet with number columns labeled number and number both columns have number rows containing numbers number number activate autofilter on the column headers for column number select from the autofilter dropdown list now only number row is displayed in the sheet examine the autofilter dropdown list for column number notice that all number values are displayed even th

## Pre-trained embeddings

Loading pretrained word vectors

### Fasttext

In [20]:
vocab = baseline.load_vocabulary(os.path.join(DIR, 'vocab_embed_fasttext.pkl'))
#print(np.random.choice(vocab, 10))
# for token in vocab:
#     print(token)

vocabulary loaded


In [21]:
"Total vocabulary: {}".format(len(vocab))

'Total vocabulary: 131563'

In [22]:
import io

def generating_embed(baseline, EMBED_DIR, EMBEDDING_DIM):
    embeddings_index = {}
    embed_path = os.path.join(EMBED_DIR, 'crawl-300d-2M.vec')
    f = open(embed_path, 'rb')
    f = io.open(embed_path, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, f.readline().split())

    vocab = baseline.load_vocabulary(os.path.join(baseline.DIR, 'vocab_embed_fasttext.pkl'))
    vocab_size = len(vocab) 

    # Initialize uniform the vector considering the Tanh activation
    embedding_matrix = np.random.uniform(-1.0, 1.0, (vocab_size, EMBEDDING_DIM))
    embedding_matrix[0, :] = np.zeros(EMBEDDING_DIM)

    loop = tqdm(f)
    loop.set_description("Loading FastText")
    for line in loop:
        tokens = line.rstrip().split(' ')
        embed = list(map(float, tokens[1:]))
        word = tokens[0]
        embeddings_index[word] = np.asarray(embed, dtype='float32')
        loop.update(1)
    f.close()
    loop.close()

    print('Total %s word vectors in FastText 42B 300d.' % len(embeddings_index))

    loop = tqdm(total=vocab_size)
    loop.set_description('Loading embedding from dataset pretrained')
    i = 0
    for word, embed in vocab.items():
        if word in embeddings_index:
            embedding_matrix[i] = embeddings_index[word]
        else:
            embedding_matrix[i] = np.asarray(embed, dtype='float32')
        loop.update(1)
        i+=1
    loop.close()
    baseline.embedding_matrix = embedding_matrix

In [23]:
%%time

generating_embed(baseline, EMBED_DIR=EMBED_DIR, EMBEDDING_DIM=EMBEDDING_DIM) # MAX_NB_WORDS=MAX_NB_WORDS

vocabulary loaded



Total 1999995 word vectors in FastText 42B 300d.



CPU times: user 2min 10s, sys: 4.72 s, total: 2min 15s
Wall time: 2min 12s


## Experiment

## Propose

https://github.com/tqtg/DuplicateBugFinder

In [24]:
from keras.initializers import RandomUniform, RandomNormal, Ones

### Embedding layer

In [25]:
from keras.constraints import MaxNorm
from keras.initializers import TruncatedNormal, RandomUniform

# Is missing the padding_idx used in pytorch
# https://pytorch.org/docs/stable/_modules/torch/nn/modules/sparse.html
# https://stackoverflow.com/questions/54824768/rnn-model-gru-of-word2vec-to-regression-not-learning
def embedding_layer(embeddings, num_words, embedding_dim, max_sequence_length, trainable, name):
    embedding_layer = Embedding(num_words,
                                  embedding_dim,
                                  name='embedding_layer_{}'.format(name),
                                  weights=[embeddings],
                                  #input_length=max_sequence_length,
                                  input_length=None,
                                  trainable=trainable)
    return embedding_layer

### DWEN model

In [26]:
from keras.layers import concatenate, Add, Lambda, merge, Average, Maximum, Subtract, \
    Average, GlobalAveragePooling1D, BatchNormalization, Activation
from keras.optimizers import Adam, Nadam
import keras.backend as K

def dwen_feature(title_feature_model, desc_feature_model, \
                  sequence_length_t, sequence_length_d, name):
    bug_t = Input(shape = (sequence_length_t, ), name = 'title_{}'.format(name))
    bug_d = Input(shape = (sequence_length_d, ), name = 'desc_{}'.format(name))
    
    # Embedding feature
    bug_t_feat = title_feature_model(bug_t)
    bug_d_feat = desc_feature_model(bug_d)
    
    bug_t_feat = GlobalAveragePooling1D()(bug_t_feat)
    bug_d_feat = GlobalAveragePooling1D()(bug_d_feat)
    
    #bug_feature_output = Add(name = 'merge_features_{}'.format(name))([bug_i_feat, bug_t_feat, bug_d_feat])
    bug_feature_output = Average(name = 'merge_features_{}'.format(name))([bug_t_feat, bug_d_feat])
    
    bug_feature_model = Model(inputs=[bug_t, bug_d], outputs=[bug_feature_output], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

def dwen_model(bug_feature_output_a, bug_feature_output_b, name):
    
    inputs = np.concatenate([bug_feature_output_a.input, bug_feature_output_b.input], -1).tolist()
    
    bug_feature_output_a = bug_feature_output_a.output
    bug_feature_output_b = bug_feature_output_b.output
    
    # 2D concatenate feature
    bug_feature_output = concatenate([bug_feature_output_a, bug_feature_output_b])
    
    hidden_layers = 2
    
    # Deep Hidden MLPs
    for _ in range(hidden_layers):
        number_of_units = K.int_shape(bug_feature_output)[1]
        bug_feature_output = Dense(number_of_units // 2)(bug_feature_output)
#         bug_feature_output = BatchNormalization()(bug_feature_output)
        bug_feature_output = Activation('relu')(bug_feature_output)
        #bug_feature_output = Dropout(.5)(bug_feature_output)
    
     # Sigmoid
    output = Dense(1, activation='sigmoid')(bug_feature_output)

    similarity_model = Model(inputs=inputs, outputs=[output], name = 'dwen_output')

    #optimizer = Nadam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=K.epsilon(), schedule_decay=0.01)
    optimizer = Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

    # setup the optimization process 
    similarity_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])
    
    return similarity_model

In [27]:
%%time

# Inspired on https://'pastebin.com/TaGFdcBA
# TODO: https://stackoverflow.com/questions/49941903/keras-compute-cosine-distance-between-two-flattened-outputs
keras.backend.clear_session()

# Clear GPU memory
# from numba import cuda
# cuda.select_device(0)
# cuda.close()

# Embeddings
desc_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.embedding_matrix), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_D, 
                              trainable=True, name='desc')
title_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.embedding_matrix), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_T, 
                              trainable=True, name='title')

# Similarity model
bug_feature_output_a = dwen_feature(title_embedding_layer, desc_embedding_layer, 
                                    MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'dwen_a')
bug_feature_output_b = dwen_feature(title_embedding_layer, desc_embedding_layer, 
                                    MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'dwen_b')
similarity_model = dwen_model(bug_feature_output_a, bug_feature_output_b, 'dwen')

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()


'''
    Configuration
'''
epochs = 100
best_loss = float('inf')
best_epoch = 0
verbose = 0
loss = 0

'''
    Experiment
'''
for epoch in range(epochs):
    batch_triplet_train, \
        train_input_sample, train_input_pos, train_input_neg, \
            train_sim = baseline.batch_iterator(baseline.train_data, baseline.dup_sets_train, bug_train_ids, batch_size, 1)
    
    num_batch = train_input_sample['title'].shape[0]
    pos = np.full((1, num_batch), 1)
    neg = np.full((1, num_batch), 0)
    train_sim = np.concatenate([pos, neg], -1)[0]
    
    title_sample_a = np.concatenate([train_input_sample['title'], train_input_sample['title']], 0)
    title_sample_b = np.concatenate([train_input_pos['title'], train_input_neg['title']], 0)
    desc_sample_a = np.concatenate([train_input_sample['description'], train_input_sample['description']], 0)
    desc_sample_b = np.concatenate([train_input_pos['description'], train_input_neg['description']], 0)
    train_batch = [title_sample_a, desc_sample_a, title_sample_b, desc_sample_b]
    
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    
    if (epoch+1 == epochs): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, _, debug = experiment.evaluate_validation_test(retrieval, verbose, bug_feature_output_a, issues_by_buckets, 
                                                        bug_train_ids, 'dwen')
        print("Epoch: {} Loss: {:.2f}, acc: {:.2f}, recall@25: {:.2f}".format(epoch+1, h[0],  h[1], recall))
    else:
        print("Epoch: {} Loss: {:.2f}, acc: {:.2f}".format(epoch+1, h[0],  h[1]))
    
    loss = h[0]
    
    if loss < best_loss:
        best_loss = loss
        best_epoch = epoch+1

experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
experiment.save_model(bug_feature_output_a, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
print('Best_epoch={}, Best_loss={:.2f}, Recall@25={:.2f}'.format(best_epoch, best_loss, recall))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title_dwen_a (InputLayer)       (None, 20)           0                                            
__________________________________________________________________________________________________
desc_dwen_a (InputLayer)        (None, 100)          0                                            
__________________________________________________________________________________________________
title_dwen_b (InputLayer)       (None, 20)           0                                            
__________________________________________________________________________________________________
desc_dwen_b (InputLayer)        (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_

In [28]:
_[:20]

['108544:111059,109674,108379,109366|94463:0.9235278144478798,96739:0.9233247712254524,93900:0.919052854180336,112422:0.9171351566910744,93307:0.9164236932992935,98768:0.9138113707304001,106589:0.9129436984658241,94948:0.9126986116170883,100511:0.9125041961669922,112201:0.9121172726154327,120337:0.9120121374726295,88894:0.911839947104454,117110:0.9115798324346542,115685:0.9115658849477768,123556:0.9115334823727608,91543:0.9106948152184486,94308:0.9100524485111237,123600:0.9099359810352325,90843:0.9098626524209976,97014:0.908508837223053,115634:0.9080507159233093,122613:0.9079797193408012,103821:0.9073458760976791,119585:0.9059890434145927,97095:0.9059795364737511,123045:0.9058895781636238,113219:0.905673585832119,91883:0.9053450971841812,115520:0.9037442803382874',
 '109674:108544,111059,108379,109366|110709:0.8292494714260101,115201:0.8244704157114029,110692:0.8200980722904205,111193:0.8181802928447723,108745:0.8159425556659698,110230:0.8156526535749435,114166:0.8145855516195297,93076

In [29]:
recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, verbose, bug_feature_output_a, issues_by_buckets, 
                                                            bug_train_ids, method='dwen')

"recall@25 last epoch:", recall

('recall@25 last epoch:', 0.22)

### Retrieval evaluation

In [30]:
print("Total of queries:", len(retrieval.test))

Total of queries: 2299


#### Getting the model trained

In [31]:
SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs))

'baseline_dwen_feature_100epochs_64batch(openoffice)'

In [32]:
model = experiment.get_model_vectorizer(path=SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)))

/usr/local/lib/python3.5/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [33]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title_dwen_a (InputLayer)       (None, 20)           0                                            
__________________________________________________________________________________________________
desc_dwen_a (InputLayer)        (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_layer_title (Embeddin (None, 20, 300)      39468900    title_dwen_a[0][0]               
__________________________________________________________________________________________________
embedding_layer_desc (Embedding (None, 100, 300)     39468900    desc_dwen_a[0][0]                
__________________________________________________________________________________________________
global_ave

In [34]:
recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, 0, model, issues_by_buckets, 
                                                                   bug_train_ids, method='dwen')

In [35]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

'data/processed/openoffice/exported_rank_baseline_dwen.txt'

In [36]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [37]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report

{'1 - recall_at_5': 0.13,
 '2 - recall_at_10': 0.17,
 '3 - recall_at_15': 0.19,
 '4 - recall_at_20': 0.2,
 '5 - recall_at_25': 0.22}